# Turkish XTTS Fine-tuning on Google Colab

This notebook guides you through fine-tuning XTTSv2 for Turkish language.

## Cell 1: Check GPU

Verify L4 GPU is active. Expected output should show "NVIDIA L4" with ~24GB memory.

In [ ]:
# Verify L4 GPU is active
!nvidia-smi

## Cell 2: Install Dependencies

In [ ]:
# Install PyTorch with CUDA 12.1
!pip install --upgrade pip
!pip install "torch==2.3.1" "torchaudio==2.3.1" --index-url https://download.pytorch.org/whl/cu121

# Install Coqui TTS and trainer
!pip install "TTS==0.22.0" "coqui-tts-trainer"

# Configure safe globals
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig

torch.serialization.add_safe_globals([
    XttsConfig, XttsAudioConfig, BaseDatasetConfig, XttsArgs
])

print("✓ Setup complete!")

## Cell 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set your dataset path
DATASET_ROOT = "/content/drive/MyDrive/xtts_tr_dataset"
METADATA_FILE = f"{DATASET_ROOT}/metadata.txt"

# Quick check
import os
print("Wav count:", len([f for f in os.listdir(f"{DATASET_ROOT}/wavs") if f.endswith(".wav")]))
print("Metadata exists:", os.path.isfile(METADATA_FILE))

## 📋 Dataset Structure Required

Before proceeding, make sure your Google Drive has this structure:

```
MyDrive/
  └── xtts_tr_dataset/
      ├── metadata.txt          ← One line per audio file
      └── wavs/
          ├── 0001.wav
          ├── 0002.wav
          ├── 0003.wav
          └── ... (more files)
```

**metadata.txt format** (pipe-separated):
```
FILE_ID|TRANSCRIPTION|TRANSCRIPTION
```

Example:
```
0001|Merhaba, benim adım Ayşe.|Merhaba, benim adım Ayşe.
0002|Bugün hava çok güzel.|Bugün hava çok güzel.
```

⚠️ **Important**: 
- File IDs in metadata.txt must match WAV filenames (without .wav extension)
- All audio should be mono, 22050 Hz, 16-bit PCM
- Each audio clip should be 3-15 seconds long
- Use clear, consistent speaker voice

In [ ]:
# Optional: Validate your dataset structure
import os
from pathlib import Path

DATASET_ROOT = "/content/drive/MyDrive/xtts_tr_dataset"
METADATA_FILE = f"{DATASET_ROOT}/metadata.txt"
WAVS_DIR = f"{DATASET_ROOT}/wavs"

print("🔍 Checking dataset structure...\n")

# Check if folders exist
if not os.path.exists(DATASET_ROOT):
    print(f"❌ Dataset folder not found: {DATASET_ROOT}")
    print("   Please create it in your Google Drive")
elif not os.path.exists(WAVS_DIR):
    print(f"❌ Wavs folder not found: {WAVS_DIR}")
    print("   Please create a 'wavs' subfolder")
elif not os.path.exists(METADATA_FILE):
    print(f"❌ metadata.txt not found: {METADATA_FILE}")
    print("   Please create metadata.txt with your transcriptions")
else:
    # Count files
    wav_files = [f for f in os.listdir(WAVS_DIR) if f.endswith(".wav")]
    
    # Read metadata
    with open(METADATA_FILE, 'r', encoding='utf-8') as f:
        metadata_lines = [line.strip() for line in f if line.strip()]
    
    print(f"✅ Dataset structure looks good!")
    print(f"   📁 Dataset root: {DATASET_ROOT}")
    print(f"   🔊 WAV files found: {len(wav_files)}")
    print(f"   📄 Metadata entries: {len(metadata_lines)}")
    
    # Check for mismatches
    metadata_ids = set()
    for line in metadata_lines:
        parts = line.split('|')
        if len(parts) >= 2:
            metadata_ids.add(parts[0])
    
    wav_ids = set(Path(f).stem for f in wav_files)
    
    missing_wavs = metadata_ids - wav_ids
    missing_metadata = wav_ids - metadata_ids
    
    if missing_wavs:
        print(f"\n⚠️  Warning: {len(missing_wavs)} files in metadata.txt but no WAV file:")
        for fid in list(missing_wavs)[:5]:
            print(f"      - {fid}.wav is missing")
    
    if missing_metadata:
        print(f"\n⚠️  Warning: {len(missing_metadata)} WAV files have no metadata entry:")
        for fid in list(missing_metadata)[:5]:
            print(f"      - {fid}.wav needs a line in metadata.txt")
    
    if not missing_wavs and not missing_metadata:
        print(f"\n🎉 Perfect! All files match. Ready to train!")
        print(f"\n📊 Sample metadata entries:")
        for line in metadata_lines[:3]:
            print(f"   {line}")

## Cell 4: Create Training Script

Run the cell below to create `train_gpt_xtts_tr.py` directly in Colab. This uses the `%%writefile` magic command to write the entire training script to a file.

In [ ]:
%%writefile train_gpt_xtts_tr.py
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# --------------------
# Logging / output
# --------------------
RUN_NAME = "XTTSv2_Turkish_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

OUT_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), "run_tr_tr", "training")

# --------------------
# Training parameters
# --------------------
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 4           # adjust if you OOM
GRAD_ACUMM_STEPS = 64    # ensure BATCH_SIZE * GRAD_ACUMM_STEPS >= ~256

# --------------------
# Dataset config (Turkish, LJSpeech-style formatter)
# --------------------
DATASET_ROOT = os.environ.get("XTTS_TR_DATASET_ROOT", "/content/drive/MyDrive/xtts_tr_dataset")
METADATA_FILE = os.environ.get("XTTS_TR_METADATA_FILE", os.path.join(DATASET_ROOT, "metadata.txt"))

config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="turkish_single_speaker",
    path=DATASET_ROOT,
    meta_file_train=METADATA_FILE,
    language="tr",
)

DATASETS_CONFIG_LIST = [config_dataset]

# --------------------
# Checkpoint paths (XTTS v2 + DVAE)
# --------------------
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2_tr_base")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK       = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE   = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

TOKENIZER_FILE_LINK   = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK  = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))

if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK],
        CHECKPOINTS_OUT_PATH,
        progress_bar=True,
    )

# --------------------
# Some short Turkish eval sentences for monitoring
# --------------------
SPEAKER_REFERENCE = [
    # any 3–6 sec clear samples from your dataset:
    # You can leave this empty; XTTS will still train, these are just for periodic audio eval.
]

LANGUAGE = config_dataset.language

# --------------------
# Early stopping (simple)
# --------------------
BEST_LOSS = None
MAX_PATIENCE = 1
CURRENT_PATIENCE = 0

def early_stopping_fn(eval_results):
    global BEST_LOSS, CURRENT_PATIENCE
    print(" > Early stopping hook")
    current_best_loss = eval_results.best_loss["eval_loss"]

    if BEST_LOSS is None:
        BEST_LOSS = current_best_loss
        return False

    if current_best_loss < BEST_LOSS:
        BEST_LOSS = current_best_loss
        CURRENT_PATIENCE = 0
        return False

    CURRENT_PATIENCE += 1
    if CURRENT_PATIENCE >= MAX_PATIENCE:
        print(" > Early stopping triggered")
        return True

    return False

# --------------------
# Main
# --------------------
def main():
    # model args (from XTTS v2 recipe, unchanged)
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6s
        min_conditioning_length=66150,   # 3s
        debug_loading_failures=False,
        max_wav_length=255995,           # ~11.6s
        max_text_length=300,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )

    audio_config = XttsAudioConfig(
        sample_rate=22050,
        dvae_sample_rate=22050,
        output_sample_rate=24000,
    )

    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="GPT XTTS v2 Turkish fine-tuning",
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=4,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        print_eval=True,
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-6,
        lr_scheduler="MultiStepLR",
        lr_scheduler_params={
            "milestones": [50000 * 18, 150000 * 18, 300000 * 18],
            "gamma": 0.5,
            "last_epoch": -1,
        },
        test_sentences=[
            {
                "text": "Merhaba, bu model Türkçe konuşmak için ince ayarlanmıştır.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "Uzun bir paragraf yerine daha doğal kısa cümleler kullanmak daha iyidir.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
        eval_split_size=0.05,
    )

    # Init XTTS GPT trainer model
    model = GPTTrainer.init_from_config(config)

    # Load dataset
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # Trainer
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
            # early_stopping_fn=early_stopping_fn,  # optional hook
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )

    trainer.fit()


if __name__ == "__main__":
    main()

## Cell 5: Start Training

In [ ]:
import os

# Set environment variables
os.environ["XTTS_TR_DATASET_ROOT"] = "/content/drive/MyDrive/xtts_tr_dataset"
os.environ["XTTS_TR_METADATA_FILE"] = "/content/drive/MyDrive/xtts_tr_dataset/metadata.txt"

# Start training
!python train_gpt_xtts_tr.py

## Cell 6: Monitor Training (Optional)

Run this in a separate cell while training is running.

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./run_tr_tr/training/

## Cell 7: Test Inference After Training

In [ ]:
import glob, os
from TTS.api import TTS

# Find latest run dir
run_root = "/content/run_tr_tr/training"
runs = sorted(glob.glob(os.path.join(run_root, "XTTSv2_Turkish_FT-*")))
assert runs, "No runs found"
model_dir = runs[-1]

model_path = os.path.join(model_dir, "best_model.pth")
config_path = os.path.join(model_dir, "config.json")

print("Using model:", model_path)

# Initialize TTS
tts = TTS(
    model_path=model_path,
    config_path=config_path,
    gpu=True,
)

# Generate sample
speaker_wav = "/content/drive/MyDrive/xtts_tr_dataset/wavs/0001.wav"

tts.tts_to_file(
    text="Merhaba, ben senin Türkçe konuşan yapay zeka asistanınım.",
    file_path="sample_tr.wav",
    speaker_wav=speaker_wav,
    language="tr",
)

print("✓ Generated: sample_tr.wav")

## Cell 8: Listen to Output

In [ ]:
from IPython.display import Audio
Audio("sample_tr.wav")

## Cell 9: Generate More Samples

In [ ]:
test_sentences = [
    "Bugün hava çok güzel.",
    "Nasıl yardımcı olabilirim?",
    "Türkçe konuşmak için ince ayarlanmış bir modelim.",
    "Yapay zeka teknolojisi hızla gelişiyor.",
]

for i, text in enumerate(test_sentences):
    output_file = f"test_{i+1}.wav"
    tts.tts_to_file(
        text=text,
        file_path=output_file,
        speaker_wav=speaker_wav,
        language="tr",
    )
    print(f"✓ Generated: {output_file}")
    display(Audio(output_file))

## Cell 10: Download Trained Model

In [ ]:
# Zip the trained model
!zip -r trained_model.zip {model_dir}

# Download to your computer
from google.colab import files
files.download('trained_model.zip')

## Tips

1. **Save checkpoints to Drive**: Modify `OUT_PATH` in training script to save to Drive
2. **Resume training**: Set `restore_path` to your checkpoint
3. **Monitor loss**: Loss should decrease to ~1.5-2.5 for good quality
4. **Adjust batch size**: If OOM, reduce `BATCH_SIZE` in script

## Expected Timeline

- Setup: 5-10 minutes
- Dataset upload: 10-30 minutes (depending on size)
- Training: 4-8 hours for 2h dataset
- Testing: 2-5 minutes

## Troubleshooting

### GPU Not Active

In [ ]:
# Force reconnect to L4
from google.colab import runtime
runtime.unassign()

### Training Too Slow

In [ ]:
# Verify GPU usage
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

### OOM Error

Reduce batch size in `train_gpt_xtts_tr.py`:
```python
BATCH_SIZE = 2  # or 1
```

---

**Pro tip**: Keep the Colab tab active and check progress every hour. Training can take 4-8 hours.